### OBJECTIVE
To develop a CNN-based model for detecting brain tumours in MRI images.

### BACKGROUND
Brain tumours are a serious medical condition that can lead to significant health consequences if not diagnosed and treated promptly. With the increasing volume of medical imaging data, deep learning models, specifically CNNs, have shown great promise in classifying medical images, including MRI scans of brain tumors. The ability to automate the detection of brain tumours can save valuable time in healthcare and improve patient outcomes.
### DATA
The dataset consists of MRI images for brain tumour classification.
https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset


### METHOD
The model preprocesses MRI images by resizing them to 150x150 pixels and normalising pixel values. The training data is augmented with rotations, shifts and flips to improve generalisation, while the validation and test data is only rescaled.

A CNN is designed with convolutional, pooling, batch normalisation and dropout layers. The model is built using the Adam optimiser and categorical cross-entropy loss. Hyperparameters such as learning rate and dropout rate are tuned using TensorBoard logging and performance is evaluated using accuracy, precision and recall metrics.

Finally, the performance of the model is evaluated using a confusion matrix and a classification report.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, AveragePooling2D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Copy the file from the source to the current directory
!cp /content/drive/MyDrive/brain-tumor-mri-dataset_merged_vordsustatud.zip /content

# Unzip the file
!unzip brain-tumor-mri-dataset_merged_vordsustatud.zip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values to [0, 1]
    rotation_range=20,          # Random rotations
    width_shift_range=0.2,      # Random horizontal shifts
    height_shift_range=0.2,     # Random vertical shifts
    shear_range=0.2,            # Shear transformations
    zoom_range=0.2,             # Random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode='nearest'         # Fill pixels after transformations
)

# Validation and test data: No augmentation, only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define directories (ensure your data is structured correctly)
train_dir = '/content/brain-tumor-mri-dataset_merged_vordsustatud/Training'
val_dir = '/content/brain-tumor-mri-dataset_merged_vordsustatud/Validation'
test_dir = '/content/brain-tumor-mri-dataset_merged_vordsustatud/Testing'

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),    # Image size must match the model input
    batch_size=32,             # Number of images per batch
    class_mode='categorical'   # Adjust for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False              # No shuffling for consistent test results
)


Found 3201 images belonging to 2 classes.
Found 421 images belonging to 2 classes.
Found 407 images belonging to 2 classes.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np


def test_confusion_matrix(model):
  predictions = model.predict(test_generator)

  predicted_classes = np.argmax(predictions, axis=1)

  true_labels = test_generator.classes
  print(test_generator.class_indices)

  accuracy = accuracy_score(true_labels, predicted_classes)
  print(f"Accuracy: {accuracy}")

  print(classification_report(true_labels, predicted_classes))

  print(confusion_matrix(true_labels, predicted_classes))

  return None



In [ ]:
import os
import time
from tensorboard.plugins.hparams import api as hp


root_logdir = os.path.join(os.curdir, "logs")
run_name = ''

def get_run_logdir():
    run_id = time.strftime("run_" + run_name + "_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [ ]:
#HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-2, 1e-3, 1e-4, 1e-1, 1e-5]))
#HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-2, 1e-3, 1e-4]))
#HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64, ]))
#HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.3, 0.5]))
#HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
#METRIC_ACCURACY = 'accuracy'

In [ ]:
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-3, 1e-4]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.5, 0.7]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
METRIC_ACCURACY = 'accuracy'

In [ ]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_LEARNING_RATE, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
def train_test_model(hparams):
    optimizer_name = hparams[HP_OPTIMIZER]
    learning_rate = hparams[HP_LEARNING_RATE]
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("unexpected optimizer name: %r" % (optimizer_name,))

    model = Sequential([
        Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', padding='same',
                            input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
        Flatten(),
        Dense(4096, activation='relu', input_dim=(227, 227, 1)),
        Dropout(hparams[HP_DROPOUT]),
        Dense(4096, activation='relu'),
        Dropout(hparams[HP_DROPOUT]),
        Dense(1000, activation='relu'),
        Dropout(hparams[HP_DROPOUT]),
        Dense(2, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

    logdir = get_run_logdir()
    model.fit(train_generator, epochs=5, validation_data=(val_generator), callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams)])  # log hparams
    accuracy = model.evaluate(test_generator)

    test_confusion_matrix(model)
    plot_model_stats(model)

    return accuracy

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    accuracy = tf.reduce_mean(accuracy)  # Convert to scalar if needed
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


session_num = 0

# for num_units in HP_NUM_UNITS.domain.values:
#     for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
for dropout_rate in HP_DROPOUT.domain.values:
  for optimizer in HP_OPTIMIZER.domain.values:
    for learning_rate in HP_LEARNING_RATE.domain.values:
        hparams = {HP_LEARNING_RATE: learning_rate,
        #HP_NUM_UNITS: num_units,
        HP_DROPOUT: dropout_rate,
        HP_OPTIMIZER: optimizer,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1
